<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/StockAndNewsAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install pandas-datareader

In [0]:
import urllib
import json
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as dr
import numpy as np
from datetime import datetime, date

In [13]:
def get_data(symbol, start, end):
  # Use IEX API for stats so we get all info we need  IEX Cloud consol 
  https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=symbol,companyName,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent
  # From here we can get the following information
  """
  symbol	string	
  companyName	string	Company name of the security
  marketcap	number	Market cap of the security calculated as shares outstanding * previous day close.
  week52high	number	
  week52low	number	
  week52change	number	Percentage change
  sharesOutstanding	number	Number of shares outstanding as the difference between issued shares and treasury shares. Investopedia
  avg30Volume	number	Average 30 day volume
  avg10Volume	number	Average 10 day volume
  float	number	Returns the annual shares outstanding minus closely held shares.
  employees	number	
  ttmEPS	number	Trailing twelve month earnings per share. Investopedia
  ttmDividendRate	number	Trailing twelve month dividend rate per share
  dividendYield	number	The ratio of trailing twelve month dividend compared to the previous day close price. The dividend yield is represented as a percentage calculated as (ttmDividendRate) / (previous day close price) Investopedia
  nextDividendDate	string	Expected ex date of the next dividend
  exDividendDate	string	Ex date of the last dividend
  nextEarningsDate	string	Expected next earnings report date
  peRatio	number	Price to earnings ratio calculated as (previous day close price) / (ttmEPS)
  beta	number	Beta is a measure used in fundamental analysis to determine the volatility of an asset or portfolio in relation to the overall market. Levered beta calculated with 1 year historical data and compared to SPY.
  day200MovingAvg	number	
  day50MovingAvg	number	
  maxChangePercent	number	
  year5ChangePercent	number	
  year2ChangePercent	number	
  year1ChangePercent	number	
  ytdChangePercent	number	
  month6ChangePercent	number	
  month3ChangePercent	number	
  month1ChangePercent	number	
  day30ChangePercent	number	
  day5ChangePercent	number	
  
  
  """
  
  
  data =  dr.DataReader(symbol, 'iex', start, end)[['close']]
  return data.rename(columns={'close': symbol})

SyntaxError: ignored

<h2>Authenticate User </h2>

In [0]:
from google.colab import auth
auth.authenticate_user()

<h3> Loading Nasdaq and Nyse shares </h3>

In [15]:
!gsutil cp gs://datascience-bucket-mm/nyse-companylist.csv /tmp/nyse.csv
!gsutil cp gs://datascience-bucket-mm/nasdaq-companylist.csv /tmp/nyse.csv  
  
# Print the result to make sure the transfer worked.
!cat /tmp/nyse.csv

Copying gs://datascience-bucket-mm/nyse-companylist.csv...
/ [1 files][392.3 KiB/392.3 KiB]                                                
Operation completed over 1 objects/392.3 KiB.                                    
Copying gs://datascience-bucket-mm/nasdaq-companylist.csv...
/ [1 files][462.5 KiB/462.5 KiB]                                                
Operation completed over 1 objects/462.5 KiB.                                    
"Symbol","Name","LastSale","MarketCap","IPOyear","Sector","industry","Summary Quote",
"YI","111, Inc.","5.56","$454.02M","2018","Health Care","Medical/Nursing Services","https://www.nasdaq.com/symbol/yi",
"PIH","1347 Property Insurance Holdings, Inc.","4.95","$29.76M","2014","Finance","Property-Casualty Insurers","https://www.nasdaq.com/symbol/pih",
"PIHPP","1347 Property Insurance Holdings, Inc.","24.65","$17.26M","n/a","Finance","Property-Casualty Insurers","https://www.nasdaq.com/symbol/pihpp",
"TURN","180 Degree Capital Corp.","1.969","$61.28M"

<h3>  Testing Historical API </h3>

In [0]:

def get_statistics(ticker):
  base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=companyName,symbol,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(symbol='AMZN')
  df = pd.read_csv(base_url)
  df['Symbol'] = ticker
  return df

def get_historical_data(ticker, start, end):
  return get_statistics(ticker)
  
  
  
  return dr.DataReader(ticker, "av-monthly", 
                       datetime.fromordinal(start.toordinal()),  # need to move to av monthly
                       datetime.fromordinal(end.toordinal()),
                       access_key='3K8QCCZYO839KMOU')

def get_date_ranges():
  end_date = date.today()
  start_date = end_date - BDay(60)
  return start_date, end_date

def test():
  start,end = get_date_ranges()
  print('start:{}, end:{}'.format(start_date, end_date))
  print(get_historical_data('AMZN', start_date, end_date))

<h3> Reading source data and computing performance </h3>

In [0]:
def get_nyse_df():
  return pd.read_csv('/tmp/nyse.csv', header=0)[['Symbol', 'Name', 'Sector', 'industry']]


In [0]:
def compute_performance(start_dt, end_dt, ticker='AMZN'):
  try:
    import time
    return get_historical_data(ticker, start_dt, end_dt)
    cols = df.columns
    #print('Start:{}, End:{}'.format(df[cols].iloc[0], df[cols].iloc[-1]))
    pcnt_df = df[cols].iloc[-1] / df[cols].iloc[0] - 1
    res = pd.DataFrame(pcnt_df, columns=['performance'])
    res['Symbol'] = ticker
    res['Latest'] = df[cols].iloc[-1]
    return res
  except Exception as e:
    print('Exception:{}'.format(str(e)))
    print('Unable to find data for {}:{}'.format(ticker,str(e)))

In [0]:
def find_best_performing(start_dt, end_dt):
  print('Finding Best Performing Stocks between:{}-{}'.format(start_dt, end_dt))
  nyse_df = get_nyse_df()
  symbols = nyse_df['Symbol'].values.tolist()
  print('Now we have to source data for:{}'.format(len(symbols)))
  dfs = (compute_performance(start_dt, end_dt, symbol) for symbol in symbols)
  filtered = (df for df in dfs if df is not None)
  all_data = pd.concat(filtered)
  return pd.merge(nyse_df, all_data, how='inner', on='Symbol' )
  

start_dt, end_dt = get_date_ranges()
perf_df = find_best_performing(start_dt, end_dt)#compute_performance(start_dt, end_dt)
print(perf_df.shape)

Finding Best Performing Stocks between:2019-04-08 00:00:00-2019-07-01
Now we have to source data for:3524


In [30]:
# Sorting 
perf_df.sort_values(by=['month3ChangePercent'], inplace=True, ascending=False)
perf_df.head(30)


,Symbol,Name,Sector,industry,companyName,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent
0,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919
12,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919
25,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919
2,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919
3,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919
4,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919
5,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919
6,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919
7,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919
8,AMZN,"Amazon.com, Inc.",Consumer Services,Catalog/Specialty Distribution,"Amazon.com, Inc.",1869.42,1745.65,0.260764,0.063389,0.040919


In [9]:
# Panda test
base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=companyName,symbol,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(symbol='AMZN')
df = pd.read_csv(base_url)
df['symbol'] = 'AMZN'
df.columns
  

Index(['companyName', 'day50MovingAvg', 'day200MovingAvg',
       'month6ChangePercent', 'month3ChangePercent', 'month1ChangePercent',
       'symbol'],
      dtype='object')